# Two Photon (Bruker) tutorial with holographic stimulation

This tutorial demonstraces how to access the *Two Photon dataset* using `pynwb`. 

This dataset contains the Two Photon imaging, holographic stimulation data and behavior measurements from the ViRMEN system.

Contents:

- [Reading an NWB file](#read-nwb)
- [Access subject and task metadata](#access-subject)
- [Access Imaging](#access-imaging)
- [Access Behavior](#access-behavior)
- [Access Stimulus](#access-stimulus)
- [View NWB files](#view-nwb)

A schematic representation where the source data is saved in NWB:

![Alt text](data_types_bruker.png)

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO

# The file path to a .nwb file
nwbfile_path = "/Volumes/t7-ssd/Pinto/nwbfiles/NCCR32_2023_02_20_Into_the_void_t_series_stim-000.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from the `date` and `time` variables from the ViRMEN file.

In [ ]:
nwbfile.session_start_time

# Access subject and task related metadata <a name="access-subject"></a>

## Access subject metadata

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

## Access ViRMEN experimental metadata (mazes table, stimulus protocol parameters)

This section demonstrates how to access the task related metadata in an NWB file.

The ViRMEN experimental metadata is stored in a [LabMetaData](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.LabMetaData) extension ([ndx-pinto-metadata](https://github.com/catalystneuro/ndx-pinto-metadata)). 

The LabMetaData](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.LabMetaData) object can be accessed as `nwbfile.lab_meta_data["LabMetaData"]`.

The parameters for the mazes is added to the `mazes` table within `nwbfile.lab_meta_data["LabMetaData"]` which can be accessed as `nwbfile.lab_meta_data["LabMetaData"].mazes`. 

Data arrays are read passively from the file. Accessing the data attribute of the `mazes` object does not read the data values, but presents an HDF5 object that can be indexed to read data. You can use the [:] operator to read the entire data array into memory as `nwbfile.lab_meta_data["LabMetaData"].mazes[:]`.

In [ ]:
nwbfile.lab_meta_data["LabMetaData"]

In [ ]:
nwbfile.lab_meta_data["LabMetaData"].mazes[:]

In [ ]:
nwbfile.lab_meta_data["LabMetaData"].stimulus_protocol[:]

## Access trials

Behavior trials are stored in `nwbfile.trials`. The `start_time` denotes the start time of each trial in seconds relative to the global session start time (using the "StartOfTrial" column from ViRMEN `.mat` file).
The `stop_time` denotes the end time of each trial in seconds relative to the global session start time
(using the "EndOfTrial" column from the ViRMEN `.mat` file).

`nwbfile.trials` can be converted to a pandas DataFrame for convenient analysis using `nwbfile.trials.to_dataframe`.


In [ ]:
trials = nwbfile.trials.to_dataframe()

trials[:10]

In [ ]:
trials[trials["trial_type"] == "right"][:3]

# Access TwoPhoton Imaging <a name="access-imaging"></a>

This section demonstraces how to access the raw Two Photon imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw TwoPhoton Imaging

The raw TwoPhoton imaging data is stored in `pynwb.ophys.TwoPhotonSeries` objects (for each channel and plane separately) which is added to `nwbfile.acquisition`. The data can be accessed as `nwbfile.acquisition['TwoPhotonSeries']`.

The data in [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
photon_series = nwbfile.acquisition['TwoPhotonSeries']

In [ ]:
# Visualize the imaging data.

from matplotlib import pyplot as plt

plt.imshow(photon_series.data[50].T, aspect="auto", cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()


In [ ]:
photon_series.rate

In [ ]:
photon_series.starting_time

## Accessing the segmentation data

The segmentation output for the Two Photon Imaging data is stored in `nwbfile.processing["ophys"]`. 

In NWB, the [PlaneSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.PlaneSegmentation) class stores the detected regions of interest in the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data. The [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation) can contain multiple `PlaneSegmentation` tables, so that we can store results of different segmentation algorithms or different segmentation classes.

We can access the plane segmentation for the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data as 
`nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentation"]`.


In [ ]:
plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentation"][:]
plane_segmentation[:10]

In [ ]:
plt.imshow(photon_series.data[50].T, aspect="auto", cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()

plt.imshow(plane_segmentation.image_mask[1].T, aspect="auto", cmap="RdYlBu_r")
plt.title("Image mask (single ROI)")
plt.show()


The summary images of the segmentation are stored in [Images](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.Images) container in NWB. 


In [ ]:
images = nwbfile.processing["ophys"]["SegmentationImages"]
images

In [ ]:
plt.imshow(photon_series.data[50].T, aspect="auto", cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()

plt.imshow(images.images["correlation"].data[:].T, aspect="auto", cmap="RdYlBu_r")
plt.title("Image Correlation")
plt.show()

plt.imshow(images.images["mean"].data[:].T, aspect="auto", cmap="RdYlBu_r")
plt.title("Image mean")
plt.show()

The fluroscence traces are stored in a [Fluorescence](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.Fluorescence) container, the raw traces can be accessed as `nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"]`.

In [ ]:
nwbfile.processing["ophys"]["Fluorescence"].roi_response_series

## Visualize raw traces

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

data = nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"].data[:, :10]
df = pd.DataFrame(data)
df["time"] = np.arange(0, data.shape[0])
df.set_index("time", inplace=True)
df.columns.name = 'ROIs'

import plotly.express as px

fig = px.line(df, facet_row="ROIs", facet_row_spacing=0.01)

# hide and lock down axes
fig.update_xaxes(visible=True, fixedrange=False)
fig.update_yaxes(visible=False, fixedrange=False)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=True,
    plot_bgcolor="white",
    margin=dict(t=10, l=10, b=10, r=10)
)

fig.show(config=dict(displayModeBar=True))

# Access Behavior <a name="access-behavior"></a>

This section demonstrates how to access behavioral data from the [pynwb.behavior](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#module-pynwb.behavior) module.

The behavior data is stored in the "behavior" processing module, which can be accessed as `nwbfile.processing["behavior"]`.


In [ ]:
nwbfile.processing["behavior"]

## Access Position

[SpatialSeries](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#pynwb.behavior.SpatialSeries) is a subclass of [TimeSeries](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.TimeSeries) that represents data in space, such as the spatial direction, e.g., of gaze or travel, or position of an animal over time.

The x, y (z) position of the animal is stored stored in `SpatialSeries` object inside the [Position](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#pynwb.behavior.Position) container.

The Position container can be accessed as `nwbfile.processing["behavior"]["Position"]`.

In [ ]:
nwbfile.processing["behavior"]["Position"]

The x, y, z position of the animal by ViRMEN iteration can be accessed as `nwbfile.processing["behavior"]["Position"]["SpatialSeries"]`.

In [ ]:
position_by_virmen = nwbfile.processing["behavior"]["Position"]["SpatialSeries"]
position_by_virmen

# Access Stimulus <a name="access-stimulus"></a>

This section demonstrates how to access the holographic stimulation data and metadata.

The metadata about the stimulus pattern can be accessed as `nwbfile.lab_meta_data["stimulus_pattern"]`.

In [ ]:
nwbfile.lab_meta_data["stimulus_pattern"]

The metadata about the laser and the spatial light modulator can be accessed as `nwbfile.devices["light_source"]` and `nwbfile.devices["spatial_light_modulator"]`. 

In [ ]:
nwbfile.devices["light_source"]

In [ ]:
nwbfile.devices["spatial_light_modulator"]

The holographic stimulation data is added to `nwbfile.stimulus['HolographicStimulationSeries']`. 

In [ ]:
nwbfile.stimulus['HolographicStimulationSeries']

In [ ]:
nwbfile.stimulus['HolographicStimulationSeries'].rois[:]

## Visualise holographic stimulation series

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

data = nwbfile.stimulus['HolographicStimulationSeries'].data[:, :10]
df = pd.DataFrame(data)
df["time"] = nwbfile.stimulus['HolographicStimulationSeries'].timestamps
df.set_index("time", inplace=True)
df.columns.name = 'Stimulated ROIs'

import plotly.express as px

fig = px.line(df, facet_row="Stimulated ROIs", facet_row_spacing=0.01)

# hide and lock down axes
fig.update_xaxes(visible=True, fixedrange=False)
fig.update_yaxes(visible=False, fixedrange=False)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=True,
    plot_bgcolor="white",
    margin=dict(t=10, l=10, b=10, r=10)
)

fig.show(config=dict(displayModeBar=True))

In [ ]:
from nwbwidgets import nwb2widget

nwb2widget(nwbfile)

We also use [Neurosift](https://github.com/flatironinstitute/neurosift), a platform for the visualization of neuroscience data in the web browser.